The following comes from Casella's Monte Carlo Statistical Methods book, section 3.2.

$$
\mathbb{E}_f [h(X)]
=
\int_{\chi} h(x) f(x) dx
$$

$\mathbb{E}_f [h(X)]$ stands for the expected value of $h(X)$, where $X$ is a random vairbale with pdf $f$.

$\chi$ denotes the sample space of of $X$. For the univariate case, $\chi \subset \mathbb{R}$.
For the multivariate case, $\chi \subset \mathbb{R}^m$, and it is said to have some volume $\int_\chi dx$.

And, last but not least, $h(x)$ is some generic function.


An estimate to this expected value can be made by drawin $m$ samples, $(X_1, \ldots , X_m)$, from the density $f$, and use the empirical average:

$$
\overline{h}_m = 
\sum^{m}_{j=1} h(x_j)
$$

Then the law of large numbers says that
$$
\lim_{m \rightarrow \infty} \overline{h}_m = \mathbb{E}_f [h(X)]
$$

And to measure the speed of convergence, we can compute the variance, if $h^2$ has a finite expectation value under $f$.
Thus, the speed fo convergence of $\overline{h}_m$ can be assessed with

$$
var(\overline{h}_m) =
\frac{1}{m} \int_\chi \left( h(x) - \mathbb{E}_f[h(X)] \right)^2
$$

Which can also be estimated with our sample $(X_1, \ldots, X_m)$ from the density $f$

$$
v_m = 
\frac{1}{m^2} \sum^{m}_{j=1} \left( h(x_j) - \overline{h}_m \right)^2
$$

Note that this estimate of the variance, $v_m$, has a factor $1/m^2$.
Typically, when estimating the variance of individual observations, we divide by $m-1$
(for an unbiased estimate of the population variance) or $m$ (for the mean of the squared deviations).
However, $v_m$ is not estimating the variance of $h(x)$ but of $\overline{h}_m$.


Another interesting thing to keep in mind is that for large $m$,
$$
\frac{ \overline{h}_m - \mathbb{E}_f [h(X)] }{ \sqrt{v_m} }
$$
is approximately distributed as $N(0,1)$.
Which leads to the construction of a convergence test and of confidence bounds on the approximation of $\mathbb{E}_f [h(X)]$.